## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unicodedata
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://www.mendocinocounty.org/community/novel-coronavirus/covid-19-case-data"

In [7]:
page = requests.get(url)

##  Parse

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", {"id": "widget_4_7164_4508"})

Get timestamp

In [10]:
date_text = content.find("p").text

AttributeError: 'NoneType' object has no attribute 'text'

In [25]:
assert "Last Updated" in date_text

In [26]:
date = re.sub("Last Updated:", "", date_text)

In [27]:
date = re.split("at", date)

In [28]:
date = re.split(" - ", date[0])

In [29]:
latest_date = pd.to_datetime(date[0]).date()

Get table

In [30]:
for tag in content.find_all(text=re.compile("Region")):
    table = tag.findParent("table")

In [31]:
tbody = table.tbody

In [32]:
row_list = tbody.find_all("tr")[1:]

In [33]:
dict_list = []

In [34]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [35]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [36]:
for row in row_list:
    cell_list = row.find_all("td")
    d = dict(
        county="Mendocino",
        area=safetxt(cell_list[0]),
        confirmed_cases=safenumber(cell_list[1]),
    )
    dict_list.append(d)

In [37]:
df = pd.DataFrame(dict_list)

In [38]:
df["county_date"] = latest_date

## Vet

In [39]:
try:
    assert not len(df) > 5
except AssertionError:
    raise AssertionError("Mendocino's area scraper has extra rows")

In [40]:
try:
    assert not len(df) < 5
except AssertionError:
    raise AssertionError("Mendocino's area scraper is missing rows")

## Export

Set date

In [41]:
tz = pytz.timezone("America/Los_Angeles")

In [42]:
today = datetime.now(tz).date()

In [43]:
slug = "mendocino"

In [44]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [45]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [46]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [47]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [48]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)